In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pickle 


update: Jan 27
    drop the auction's price'... how would a user know that? they only jknow the retail price.

In [2]:
with open('df1_raw.pkl', 'r') as picklefile:
    df1_raw = pickle.load(picklefile)
    
with open('lemons.pkl', 'r') as picklefile:
    lemons = pickle.load(picklefile)

In [3]:
lemons

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
12,13,1,12/14/2009,ADESA,2005,4,DODGE,CARAVAN GRAND FWD V6,SE,MINIVAN 3.3L,...,8523.0,10124.0,NaN,NaN,19638,33619,FL,5400.0,0,1623
25,26,1,12/21/2009,ADESA,2004,5,MERCURY,SABLE,LS,4D SEDAN LS PREMIUM,...,6600.0,8074.0,NaN,NaN,5546,33619,FL,4500.0,0,754
31,32,1,12/28/2009,ADESA,2002,7,FORD,EXPLORER 2WD V6,Spo,2D SUV 4.0L SPORT,...,5622.0,6773.0,NaN,NaN,19638,33619,FL,4900.0,0,1220
35,36,1,12/28/2009,ADESA,2001,8,FORD,WINDSTAR FWD V6,LX,PASSENGER 3.8L LX,...,4407.0,5530.0,NaN,NaN,19638,33619,FL,3600.0,0,2322
39,40,1,1/4/2010,ADESA,2005,5,FORD,MUSTANG V6,Bas,2D COUPE,...,11047.0,13224.0,NaN,NaN,19638,33619,FL,8500.0,0,728
41,42,1,1/4/2010,ADESA,2005,5,NISSAN,SENTRA,Bas,4D SEDAN 1.8,...,8115.0,9156.0,NaN,NaN,19638,33619,FL,5200.0,0,594
45,46,1,1/11/2010,ADESA,2004,6,NISSAN,XTERRA 2WD V6,SE,4D SPORT UTILITY,...,10916.0,12432.0,NaN,NaN,19619,33619,FL,7100.0,0,569
47,48,1,1/11/2010,ADESA,2002,8,TOYOTA,COROLLA,CE,4D SEDAN CE,...,5516.0,7088.0,NaN,NaN,19619,33619,FL,3900.0,0,869
48,49,1,1/11/2010,ADESA,2004,6,FORD,FOCUS,SE,4D SEDAN SE,...,6112.0,7097.0,NaN,NaN,19619,33619,FL,4100.0,0,1155
49,50,1,1/11/2010,ADESA,2006,4,PONTIAC,G6 V6,GT,4D SEDAN GT,...,10660.0,12253.0,NaN,NaN,19619,33619,FL,7500.0,0,1003


In [4]:
# create a column where make and model are concat

df1_raw["make_model"] = df1_raw["Make"].map(str) +" " + df1_raw["Model"]

# Extracting year and month the car was purhcased

def clean_month(x):
    x = x.split("/")
    x= x[0]
    return x

def clean_year(x):
    x= x.split("/")
    x = x[2]
    return x


df1_raw["month"] = [clean_month(x) for x in df1_raw["PurchDate"]]
df1_raw["year"] = [clean_year(x) for x in df1_raw["PurchDate"]]


In [5]:
df1_raw["WheelType"].unique()

array(['Alloy', 'Covers', nan, 'Special'], dtype=object)

In [6]:
# the first set from first half of the df
df2 = df1_raw.ix[:, 1:18]
df2= df2.drop(['PurchDate','Trim','WheelTypeID', 'Model', 'SubModel','TopThreeAmericanName','VehicleAge'], 1)

#second half of the dataset. All to get rid of middle prices
df2a = df1_raw.ix[:,24:]
df2a= df2a.drop(['VehBCost','MMRCurrentRetailCleanPrice','PRIMEUNIT','AUCGUART', 'ones','BYRNO','VNZIP1'], 1)
df2a.head()

,MMRCurrentRetailAveragePrice,VNST,IsOnlineSale,WarrantyCost,make_model,month,year
0,11597.0,FL,0,1113,MAZDA MAZDA3,12,2009
1,11374.0,FL,0,1053,DODGE 1500 RAM PICKUP 2WD,12,2009
2,7146.0,FL,0,1389,DODGE STRATUS V6,12,2009
3,4375.0,FL,0,630,DODGE NEON,12,2009
4,6739.0,FL,0,1020,FORD FOCUS,12,2009


In [7]:
df3_set = pd.concat([df2, df2a], axis=1, join='inner')
df3_set.columns

Index([u'IsBadBuy', u'Auction', u'VehYear', u'Make', u'Color', u'Transmission',
       u'WheelType', u'VehOdo', u'Nationality', u'Size',
       u'MMRCurrentRetailAveragePrice', u'VNST', u'IsOnlineSale',
       u'WarrantyCost', u'make_model', u'month', u'year'],
      dtype='object')

In [7]:
# clean make and model into one category and drop the last 
def clean_make(x):
    x = x.split(" ")
    x= x[0] + " " + x[1]
    return x

df3_set["model"] = [clean_make(x) for x in df3_set["make_model"]]
df3_set.head()

,IsBadBuy,Auction,VehYear,Make,Color,Transmission,WheelType,VehOdo,Nationality,Size,MMRCurrentRetailAveragePrice,VNST,IsOnlineSale,WarrantyCost,make_model,month,year,model
0,0,ADESA,2006,MAZDA,RED,AUTO,Alloy,89046,OTHER ASIAN,MEDIUM,11597.0,FL,0,1113,MAZDA MAZDA3,12,2009,MAZDA MAZDA3
1,0,ADESA,2004,DODGE,WHITE,AUTO,Alloy,93593,AMERICAN,LARGE TRUCK,11374.0,FL,0,1053,DODGE 1500 RAM PICKUP 2WD,12,2009,DODGE 1500
2,0,ADESA,2005,DODGE,MAROON,AUTO,Covers,73807,AMERICAN,MEDIUM,7146.0,FL,0,1389,DODGE STRATUS V6,12,2009,DODGE STRATUS
3,0,ADESA,2004,DODGE,SILVER,AUTO,Alloy,65617,AMERICAN,COMPACT,4375.0,FL,0,630,DODGE NEON,12,2009,DODGE NEON
4,0,ADESA,2005,FORD,SILVER,MANUAL,Covers,69367,AMERICAN,COMPACT,6739.0,FL,0,1020,FORD FOCUS,12,2009,FORD FOCUS


In [9]:
import pickle
with open('original.pkl', 'w') as picklefile:
    pickle.dump(df3_set, picklefile)

In [11]:
df3_set.ix[38298,:]

IsBadBuy                                           1
Auction                                      MANHEIM
VehYear                                         2006
Make                                           MAZDA
Color                                           GREY
Transmission                                    AUTO
WheelType                                      Alloy
VehOdo                                         71739
Nationality                              OTHER ASIAN
Size                                       SMALL SUV
MMRCurrentRetailAveragePrice                   10304
VNST                                              TX
IsOnlineSale                                       0
WarrantyCost                                     920
make_model                      MAZDA TRIBUTE 2WD 4C
month                                             10
year                                            2010
model                                  MAZDA TRIBUTE
Name: 38298, dtype: object

In [9]:
df3_set = df3_set.dropna()
df3_set.head()

,IsBadBuy,Auction,VehYear,Make,Color,Transmission,WheelType,VehOdo,Nationality,Size,MMRCurrentRetailAveragePrice,VNST,IsOnlineSale,WarrantyCost,make_model,month,year,model
0,0,ADESA,2006,MAZDA,RED,AUTO,Alloy,89046,OTHER ASIAN,MEDIUM,11597.0,FL,0,1113,MAZDA MAZDA3,12,2009,MAZDA MAZDA3
1,0,ADESA,2004,DODGE,WHITE,AUTO,Alloy,93593,AMERICAN,LARGE TRUCK,11374.0,FL,0,1053,DODGE 1500 RAM PICKUP 2WD,12,2009,DODGE 1500
2,0,ADESA,2005,DODGE,MAROON,AUTO,Covers,73807,AMERICAN,MEDIUM,7146.0,FL,0,1389,DODGE STRATUS V6,12,2009,DODGE STRATUS
3,0,ADESA,2004,DODGE,SILVER,AUTO,Alloy,65617,AMERICAN,COMPACT,4375.0,FL,0,630,DODGE NEON,12,2009,DODGE NEON
4,0,ADESA,2005,FORD,SILVER,MANUAL,Covers,69367,AMERICAN,COMPACT,6739.0,FL,0,1020,FORD FOCUS,12,2009,FORD FOCUS


In [10]:
df3_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69494 entries, 0 to 72982
Data columns (total 18 columns):
IsBadBuy                        69494 non-null int64
Auction                         69494 non-null object
VehYear                         69494 non-null int64
Make                            69494 non-null object
Color                           69494 non-null object
Transmission                    69494 non-null object
WheelType                       69494 non-null object
VehOdo                          69494 non-null int64
Nationality                     69494 non-null object
Size                            69494 non-null object
MMRCurrentRetailAveragePrice    69494 non-null float64
VNST                            69494 non-null object
IsOnlineSale                    69494 non-null int64
WarrantyCost                    69494 non-null int64
make_model                      69494 non-null object
month                           69494 non-null object
year                           

In [11]:
holder = list(df3_set["Size"].unique())

In [12]:
holder

['MEDIUM',
 'LARGE TRUCK',
 'COMPACT',
 'LARGE',
 'VAN',
 'MEDIUM SUV',
 'LARGE SUV',
 'SPECIALTY',
 'SPORTS',
 'CROSSOVER',
 'SMALL SUV',
 'SMALL TRUCK']

In [13]:
df3_set.describe()

,IsBadBuy,VehYear,VehOdo,MMRCurrentRetailAveragePrice,IsOnlineSale,WarrantyCost
count,69494.000000,69494.000000,69494.000000,69494.000000,69494.000000,69494.000000
mean,0.096512,2005.355599,71476.487092,8801.659568,0.024822,1276.114240
std,0.295294,1.725448,14564.526298,3084.773443,0.155584,597.527202
min,0.000000,2001.000000,5368.000000,0.000000,0.000000,462.000000
25%,0.000000,2004.000000,61838.000000,6570.000000,0.000000,837.000000
50%,0.000000,2005.000000,73338.500000,8756.000000,0.000000,1169.000000
75%,0.000000,2007.000000,82394.000000,10927.000000,0.000000,1623.000000
max,1.000000,2010.000000,115717.000000,39080.000000,1.000000,7498.000000


In [14]:
df3_set = df3_set.drop(["make_model","WheelType", "Transmission","IsOnlineSale"],1 )

In [15]:
df3_set.head()

,IsBadBuy,Auction,VehYear,Make,Color,VehOdo,Nationality,Size,MMRCurrentRetailAveragePrice,VNST,WarrantyCost,month,year,model
0,0,ADESA,2006,MAZDA,RED,89046,OTHER ASIAN,MEDIUM,11597.0,FL,1113,12,2009,MAZDA MAZDA3
1,0,ADESA,2004,DODGE,WHITE,93593,AMERICAN,LARGE TRUCK,11374.0,FL,1053,12,2009,DODGE 1500
2,0,ADESA,2005,DODGE,MAROON,73807,AMERICAN,MEDIUM,7146.0,FL,1389,12,2009,DODGE STRATUS
3,0,ADESA,2004,DODGE,SILVER,65617,AMERICAN,COMPACT,4375.0,FL,630,12,2009,DODGE NEON
4,0,ADESA,2005,FORD,SILVER,69367,AMERICAN,COMPACT,6739.0,FL,1020,12,2009,FORD FOCUS


In [16]:
# columns that need dummy variables

col_list = ["Auction", "VehYear", "Color", "Size", "VNST","Nationality", "model", "month" ]
df_dict = {}

for i in col_list:
    df_dict[i] = pd.get_dummies(df3_set[i])

frames = df_dict.values()
result = pd.concat(frames, axis=1)
result.head()

,ADESA,MANHEIM,OTHER,1,10,11,12,2,3,4,...,OK,OR,PA,SC,TN,TX,UT,VA,WA,WV
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# df4 includes all the desired columns and the dummies
df4 = pd.concat([df3_set, result], axis=1, join='inner')
df4.head()


,IsBadBuy,Auction,VehYear,Make,Color,VehOdo,Nationality,Size,MMRCurrentRetailAveragePrice,VNST,...,OK,OR,PA,SC,TN,TX,UT,VA,WA,WV
0,0,ADESA,2006,MAZDA,RED,89046,OTHER ASIAN,MEDIUM,11597.0,FL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,ADESA,2004,DODGE,WHITE,93593,AMERICAN,LARGE TRUCK,11374.0,FL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,ADESA,2005,DODGE,MAROON,73807,AMERICAN,MEDIUM,7146.0,FL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,ADESA,2004,DODGE,SILVER,65617,AMERICAN,COMPACT,4375.0,FL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,ADESA,2005,FORD,SILVER,69367,AMERICAN,COMPACT,6739.0,FL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df3_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69494 entries, 0 to 72982
Data columns (total 14 columns):
IsBadBuy                        69494 non-null int64
Auction                         69494 non-null object
VehYear                         69494 non-null int64
Make                            69494 non-null object
Color                           69494 non-null object
VehOdo                          69494 non-null int64
Nationality                     69494 non-null object
Size                            69494 non-null object
MMRCurrentRetailAveragePrice    69494 non-null float64
VNST                            69494 non-null object
WarrantyCost                    69494 non-null int64
month                           69494 non-null object
year                            69494 non-null object
model                           69494 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 8.0+ MB


In [19]:
df5_part = df3_set.loc[:,["IsBadBuy", "VehOdo","MMRCurrentRetailAveragePrice","WarrantyCost" ]]
df5_part.head()

,IsBadBuy,VehOdo,MMRCurrentRetailAveragePrice,WarrantyCost
0,0,89046,11597.0,1113
1,0,93593,11374.0,1053
2,0,73807,7146.0,1389
3,0,65617,4375.0,630
4,0,69367,6739.0,1020


In [20]:
#combine the two halves of the dataset's columns
df5 = pd.concat([df5_part, result], axis=1, join='inner')

# reset index
df5= df5.reset_index()

df5.head()

,index,IsBadBuy,VehOdo,MMRCurrentRetailAveragePrice,WarrantyCost,ADESA,MANHEIM,OTHER,1,10,...,OK,OR,PA,SC,TN,TX,UT,VA,WA,WV
0,0,0,89046,11597.0,1113,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,93593,11374.0,1053,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,73807,7146.0,1389,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,65617,4375.0,630,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,69367,6739.0,1020,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn import preprocessing
def norm(col):
    df = df5[col]
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(df)
    df_normalized = pd.DataFrame(np_scaled)
    df5[col] = df_normalized

cols = ["VehOdo","MMRCurrentRetailAveragePrice", "WarrantyCost"]

for item in cols:
    norm(item)


/Users/ellenkim/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/ellenkim/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/ellenkim/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/ellenki

In [22]:
#np_scaled.transform(np_scaled)

In [23]:
df5.head()

,index,IsBadBuy,VehOdo,MMRCurrentRetailAveragePrice,WarrantyCost,ADESA,MANHEIM,OTHER,1,10,...,OK,OR,PA,SC,TN,TX,UT,VA,WA,WV
0,0,0,0.758303,0.296750,0.092524,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0.799509,0.291044,0.083997,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0.620205,0.182856,0.131751,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,0.545986,0.111950,0.023877,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,0.579969,0.172441,0.079306,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
import pickle
with open('with_index.pkl', 'w') as picklefile:
    pickle.dump(df5, picklefile)

In [25]:
df5 = df5.drop("index", 1)


In [27]:
df_col = list(df5.columns)

In [28]:
df_col = pd.DataFrame(df_col).T

In [26]:
import pickle
with open('df_columns.pkl', 'w') as picklefile:
    pickle.dump(df_col, picklefile)

In [26]:
import pickle
with open('cleanv1.pkl', 'w') as picklefile:
    pickle.dump(df5, picklefile)

In [27]:
import pickle
with open('drop_wt_trans.pkl', 'w') as picklefile:
    pickle.dump(df5, picklefile)